In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import matplotlib.pyplot as plt
import penn
import promonet
import torch
import torchaudio


In [ ]:
gpu = 0
dataset = 'vctk'

# Get cache directory
directory = promonet.CACHE_DIR / 'vctk'

# Get text and audio files for this speaker
audio_files = sorted(list(directory.rglob('*.wav')))
audio_files = [file for file in audio_files if '-' in file.stem]


In [ ]:
for file in audio_files[6877:6881]:
    audio = promonet.load.audio(file)
    pitch, periodicity, loudness, ppg = promonet.preprocess.from_audio(audio, gpu=1)
    fig = promonet.plot.from_features(
        audio,
        *promonet.load.features(file.parent / file.stem),
        pitch.cpu(),
        periodicity.cpu(),
        loudness.cpu(),
        ppg.cpu())


In [ ]:
# w/ clean-up
numfiles = 6900
penn.from_files_to_files(
    audio_files[:numfiles],
    hopsize=promonet.convert.samples_to_seconds(promonet.HOPSIZE),
    fmin=promonet.FMIN,
    fmax=promonet.FMAX,
    batch_size=2048,
    center='half-hop',
    interp_unvoiced_at=promonet.VOICING_THRESOLD,
    gpu=gpu)


In [ ]:
batch_pitch = torch.load('pitch.pt')
batch_periodicity = torch.load('periodicity.pt')
batch_lengths = [264, 334, 217, 283, 217, 202, 198, 198, 117, 195, 122, 325, 195, 400, 314, 314, 209]


In [ ]:
x = 0
for length in batch_lengths[:-2]:
    test_periodicity = batch_periodicity[0, x:x + length]
    plt.plot(torch.arange(x, x + length), test_periodicity)
    x += length
    plt.axvline(x)
